**<center><span style="font-size:88px;">openrouteservice - route the world dynamically</span></center>**

# Fact Sheet

- **openrouteservice** is a project of the KTS backed HeiGIT institute at the University of Heidelberg
- started in 2008, **openrouteservice** was one of the first routing API's
- entirely **open-source**, backend written in **Java**
- based on **OpenStreetMap** data
- 2500 requests per day **for free**

### Available API's
> **Directions**

> **Isochrones**

> **Geocoding**

> **Matrix**

> **Places**

# <center style="font-size: 200px">Architecture</center>

In [1]:
import folium
import branca
from folium.plugins import MeasureControl
from shapely import geometry
from IPython.display import HTML, display, clear_output
from ipywidgets import widgets, interact, interactive
import json
from urllib.parse import urlencode
import requests

from pprint import pprint
from numpy import average

api_key = json.load(open('token.json'))

In [2]:
def createBaseMap(width, height, coords=(49.406486, 8.683481), zoom=15, map_provider='stamenwatercolor'):
    fig = folium.Figure(width=width, height=height)
    m = folium.Map(tiles=map_provider, location=coords, zoom_start=zoom)
    
    m.add_child(MeasureControl())
#     m.add_child(folium.LayerControl())
    
    return fig, m

## Directions API

Classical routing API, based on the excellent [GraphHopper](https://github.com/graphhopper/graphhopper) stack.

<div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
<h3>Features</h3>
<br>
<div style=" border-right: 1px solid black">
<ul>
    <li>10 routing profiles:
        <ul>
            <li>2 car: `driving-car`, `driving-hgv`
            <li>5 cycling, incl. `*-mountain`, `*-safe`
            <li>2 walking: `foot-walking`, `foot-hiking`
            <li><strong>`wheelchair`</strong>
        </ul>
    <li>Output formats: `geojson`, `gpx`
    <li>13 languages for instructions
    <li>Returns extra way information for route segments, e.g. steepness, surface, tollways
</ul>
</div>
</div>

<div style="margin:0px 20px; width: 50%; display: inline-block;vertical-align: top">
<h3>Functionalities</h3>
<ul style="line-height: 30px">
    <li>`bearing`, `radiuses`: Control input point snapping behavior
    <li>`continue_straight`: Avoid u-turns, even if faster
    <li>`avoid_*`: Avoid features, e.g. ferry, **countries**, **polygons**
    <li>vehicle dimensions: use OSM dimension tags to route compliantly
    <li>**green**/**quiet**: prefer green and/or quiet routes
</ul>
</div>

## Avoid features for `cycling-regular`

In [3]:
widget_hills = widgets.ToggleButtons(
                    options=['hills', 'unpavedroads', 'no restriction'],
                    description='Avoid:',
                    disabled=False,
                    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                    )

def on_change_avoid(change):    
    route = avoidHills(change['new'])
    bbox = route['bbox']
    bbox_centroid = (average([bbox[1],bbox[3]]),average([bbox[0],bbox[2]]))
    route_properties = route['features'][0]['properties']
    
    # Clear all output
    clear_output()
    
    fig1, map1 = createBaseMap('100%', '400', map_provider='OpenStreetMap', coords=bbox_centroid, zoom=12)
    popup_html = """<strong>duration: </strong>{0:.1f} mins<br>
                    <strong>distance: </strong>{1:.3f} km""".format(route_properties['summary'][0]['duration'] / 60,
                                                           route_properties['summary'][0]['distance'] / 1000)
    
    iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
    popup = folium.Popup(iframe, max_width=300)
    
    gj = folium.features.GeoJson(route)
    gj.add_child(popup)
    gj.add_to(map1)
    map1.add_to(fig1)
    
    display(widget_hills)
    display(fig1)
    
        
widget_hills.observe(on_change_avoid, names='value')

In [12]:
import openrouteservice as ors

ors_key = api_key["ORS"] # comes from local file
clnt = ors.Client(key=ors_key) # set up client
# This function is called by the widget's event callback
def avoidHills(mode):
    params = {'coordinates': [[8.681602, 49.422141],[8.737221, 49.49333]],
              'profile': 'cycling-regular',
              'format_out': 'geojson'}
    if mode != 'no restriction':
        params['options'] = {'avoid_features': mode}
    route = clnt.directions(**params)
    return route

widget_hills

ToggleButtons(button_style='info', description='Avoid:', options=('hills', 'unpavedroads', 'no restriction'), …

## Avoid countries for `driving-hgv`

In [4]:
widget_country = widgets.SelectMultiple(
    options={'DE':75, 'CH': 193, 'AT': 11, 'None': 'None'},
    value=[75],
    rows=4,
    description='Countries',
    disabled=False
)

def on_change_country(change):
    route = avoidCountry(change['new'])
    bbox = route['bbox']
    bbox_centroid = (average([bbox[1],bbox[3]]),average([bbox[0],bbox[2]]))
    route_properties = route['features'][0]['properties']
    
    # Clear all output
    clear_output()
    
    fig1, map1 = createBaseMap('100%', '400', map_provider='OpenStreetMap', coords=bbox_centroid, zoom=6)
    popup_html = """<strong>duration: </strong>{0:.1f} mins<br>
                    <strong>distance: </strong>{1:.3f} km""".format(route_properties['summary'][0]['duration'] / 60,
                                                           route_properties['summary'][0]['distance'] / 1000)
    
    iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
    popup = folium.Popup(iframe, max_width=300)
    
    gj = folium.features.GeoJson(route)
    gj.add_child(popup)
    gj.add_to(map1)
    map1.add_to(fig1)
    
    display(widget_country)
    display(fig1)
    
        
widget_country.observe(on_change_country, names='value')

In [13]:
import openrouteservice as ors

ors_key = api_key["ORS"] # comes from local file
clnt = ors.Client(key=ors_key) # set up client
# This function is called by the widget's event callback
def avoidCountry(country):
    country = '|'.join([str(c) for c in country])
    params = {'coordinates': [[8.706665, 49.50381],[9.217529, 45.521744]],
              'profile': 'driving-hgv',
              'format_out': 'geojson',}
    if country != 'None':
        params['options'] = {'avoid_countries': country}
    route = clnt.directions(**params)
    return route

widget_country

SelectMultiple(description='Countries', index=(1,), options={'DE': 75, 'CH': 193, 'AT': 11, 'None': 'None'}, r…

## Isochrones API

<div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
<h3>Features</h3>
<br>
<div style=" border-right: 1px solid black">
<ul>
    <li>9 routing profiles:
        <ul>
            <li>2 car: `driving-car`, `driving-hgv`
            <li>5 cycling, incl. `*-mountain`, `*-safe`
            <li>2 walking: `foot-walking`, `foot-hiking`
        </ul>
    <li>Query up to 5 locations and 10 intervals in one request
</ul>
</div>
</div>

<div style="margin:0px 20px; width: 50%; display: inline-block;vertical-align: top">
<h3>Functionalities</h3>
<ul style="line-height: 30px">
    <li>**`attributes`**: Returns e.g. total population within isochrone<sup>1</sup>
    <li>plus similar functionalities as **Directions* API
</ul>
</div>

<br>
<br>

<small><sup>1</sup> Global data coverage by [EU GSH](https://ghsl.jrc.ec.europa.eu/about.php)</small>

## Geocoding API

Fork of **[Pelias](https://github.com/pelias/pelias)** geocoding engine.

<div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
<h3>Features</h3>
<br>
<div style=" border-right: 1px solid black">
<ul>
    <li>3 endpoints:
        <ul>
            <li>`geocoding/search?`
            <li>`geocoding/reverse?`
            <li>`geocoding/autocomplete?`
        </ul>
    <li>4 data sources:
        <ul>
            <li>**WhosOnFirst**: Global administrative boundaries
            <li>**OSM**
            <li>**openaddresses**: ~ 500 Mio addresses
            <li>**geonames**: Additional place names and locations
</ul>
</div>
</div>

<div style="margin:0px 20px; width: 50%; display: inline-block;vertical-align: top">
<h3>Functionalities</h3>
<ul style="line-height: 30px">
    <li>**libpostal**: address parser and normalizer
    <li>**placeholder**: address parser for unstructured text
    <li>**interpolation**: attempts interpolation for unknown house numbers
</ul>
<br>
These extra services are stand-alone API's and can be used in any application.
</div>

## Matrix API

<div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
<h3>Features</h3>
<br>
<ul>
    <li>Calculates OD matrices for up to **50x50 locations**
    <li>For all 9 profiles
    <li>Query up to 5 locations and 10 intervals in one request
</ul>
</div>

## Infrastructure

**openrouteservice** offers multiple clients to consume our API's:
<div style="display: block">
    <div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
        <img src="../img/python.png" style="width:200px; height:200px">
    </div>
    <div style="margin:0px 20px; width: 40%; display: inline-block; vertical-align: top;">
        Excellent text
    </div>
</div>

# You know what's on for dinner?

Let's see how OSM and Yelp make the restaurant choice a little easier:

- Find restaurants within 10 min walking distance from SOTM location
- Filter restaurants with Yelp rating >= 4
- Create widget to filter by restaurant category

## Set up *openrouteservice* client

1. Sign up on our [homepage](https://openrouteservice.org/sign-up/) 
2. Create API key on our [developer portal](https://openrouteservice.org/dev/#/home)
3. Review our [ToS](https://openrouteservice.org/terms-of-service/)
4. `pip install openrouteservice`
5. Fire away!

Here we:
- _**Geocode**_ the SOTM official address
- Create an _**isochrone**_ with 10 mins walking radius

In [14]:
import openrouteservice as ors

clnt = ors.Client(key='58d904a497c67e00015b45fc3535fd17dd8948bb8cb330e216534497')

params_geocode = {'text': 'Politecnico di Milano ‐ Piazza Leonardo da Vinci, Milan, Lombardy, Italy'}

sotm_location = clnt.pelias_search(**params_geocode)['features'] # Geocode from official address
sotm_isochrone = clnt.isochrones(locations=sotm_location[0]['geometry']['coordinates'], # Calculate isochrones
                                 intervals=[600],
                                 segments=600,
                                 profile='foot-walking')

In [8]:
x, y = sotm_location[0]['geometry']['coordinates']
def createWidgetBaseMap(width, height, include=True, map_provider='stamenwatercolor'):
    bar_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/bar.png',
                                       icon_size=(30, 30))
    sotm_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/sotm_map_marker.png', icon_size=(58, 70))

    fig = folium.Figure(width=width, height=height)
    m = folium.Map(tiles=map_provider,location=(y, x), zoom_start=15)
    folium.map.Marker([y, x], icon=sotm_icon).add_to(m)
    if include:
        folium.features.GeoJson(sotm_isochrone).add_to(m)
    
    m.add_child(MeasureControl())
    
    return fig, m

## Set up a basic `folium` map

- The `createBaseMap` factory sets up the initial map, so we start fresh every time

In [15]:
fig, m = createWidgetBaseMap('100%', 600) # Call map factory with figure dimensions
fig.add_child(m)
fig

## Request restaurant POI's

- `/pois?` endpoint takes GeoJSON
- we can pipe the GeoJSON output of `/isochrones?` directly to `/pois?` to limit the query to the isochrones geometry

In [16]:
param_pois = {'request': 'pois',
            'geojson': sotm_isochrone['features'][0]['geometry'],
            'filter_category_ids': [570], # ID list: https://github.com/GIScience/openrouteservice-docs#sustenance--560
            'sortby': 'distance'}

sotm_restaurants = clnt.places(**param_pois)['features']
display(HTML('<br><br>Amount of restaurants within 10 mins walking radius: <strong>{}</strong>'.format(len(sotm_restaurants))))

## Yelp rating

- use the [Yelp businesses API](https://www.yelp.de/developers/documentation/v3/business) to query the reviews of our restaurants
- extract `name`, `rating` and `url` to feature popup

In [10]:
yelp_url = 'https://api.yelp.com/v3/businesses/search?' # Create a Yelp account and a API key: https://www.yelp.com/developers
yelp_header = {'Authorization': api_key['Authorization']} # use secret from file: {'Authorization': 'Bearer MyAPIkey'}
yelp_params = {'limit': 1}

def yelp_response(**kwargs):
    yelp_params.update({'term': r_name,
                        'longitude': r_coords[0],
                        'latitude': r_coords[1]})
    # Concat params to URL encoded string
    yelp_url_params = requests.utils.unquote_unreserved(urlencode(list(yelp_params.items())))
    try:
        yelp_response = requests.get(yelp_url + yelp_url_params,
                                     headers=yelp_header).json()['businesses'][0]
    except:
        return False

    if yelp_response['rating'] < 4.0:
        return False
    
    return yelp_response

In [ ]:
import branca

fig, m = createWidgetBaseMap('100%', '500')
for restaurant in sotm_restaurants:
    r_name = restaurant['properties']['osm_tags'].get('name', '')
    if r_name != '':
        r_coords = restaurant['geometry']['coordinates']
        yelp_return = yelp_response(r_name=r_name, # Call hidden function to retrieve Yelp results with rating >= 4 stars
                                    r_coords=r_coords)
        if yelp_return:
            restaurant['yelp'] = yelp_return
            restaurant_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/restaurant2.png',
                                                   icon_size=(50, 50))
            popup_html = '<h4>{0}</h4><strong><a href="{1}" target="_blank">URL</a><br>Stars: </strong>{2}'.format(r_name, yelp_return['url'], yelp_return['rating'])
            iframe = branca.element.IFrame(html=popup_html, width=300, height=100)
            popup = folium.Popup(iframe, max_width=300)
            folium.map.Marker(list(reversed(r_coords)), icon=restaurant_icon, popup=popup).add_to(m)
fig.add_child(m)
fig

In [11]:
# Define change function for drop-down
def on_category_change(change):
    # First we need to clear_output() entirely and then re-draw all components
    clear_output()
    
    x, y = sotm_location[0]['geometry']['coordinates']
    fig2, map2 = createWidgetBaseMap('100%', 600, include=False)
    
    for restaurant in sotm_restaurants:
        if restaurant.get('yelp', '') != '':
            if restaurant['yelp']['categories'][0]['title'] == change['new']:
                restaurant_coords = restaurant['geometry']['coordinates']
                clnt = ors.Client(key='58d904a497c67e00015b45fc3535fd17dd8948bb8cb330e216534497')
                route = clnt.directions(coordinates=[[x,y],
                                                     restaurant_coords
                                                    ],
                                       format_out='geojson',
                                       profile='foot-walking',
                                       geometry_format='geojson')
                
                route_properties = route['features'][0]['properties']
                folium.features.GeoJson(route).add_to(map2)

                restaurant_icon = folium.features.CustomIcon('https://openrouteservice.org/wp-content/uploads/2017/07/restaurant2.png',
                                                             icon_size=(50, 50))
                popup_html = """<h4>{0}</h4>
                                <strong><a href="{1}" target="_blank">URL</a></strong><br>
                                <strong>Category: </strong>{2}<br>
                                <strong>Walking time: </strong>{3:.2f} mins<br>
                                <strong>Reviews: </strong>{4}<br>                                
                                <strong>Stars: </strong>{5}""".format(restaurant['yelp']['name'],
                                                              restaurant['yelp']['url'],
                                                              change['new'],
                                                              route_properties['summary'][0]['duration'] / 60,
                                                              restaurant['yelp']['review_count'],
                                                              restaurant['yelp']['rating'])
                
                iframe = branca.element.IFrame(html=popup_html, width=300, height=150)
                popup = folium.Popup(iframe, max_width=300)
                folium.map.Marker(list(reversed(restaurant_coords)),
                                  icon=restaurant_icon,
                                  popup=popup,
                                 ).add_to(map2)
    fig2.add_child(map2)
    # Now we re-add the widgets to the cleared output area
    display(widget_dd)
    display(fig2)

## IPython widget

The final product is a drop-down widget to query the directions to nearby restaurants by restaurant category:

In [ ]:
category_titles = []
for restaurant in sotm_restaurants:
    if restaurant.get('yelp', '') != '':
        category_titles.append(restaurant['yelp']['categories'][0]['title'])

widget_dd = widgets.Dropdown(options=set(category_titles)) # Create widget with categories in drop-down
widget_dd.observe(on_category_change, names='value') # Hidden method is called by the listener        
display(widget_dd)